🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Cải thiện Decision Tree đơn lẻ bằng cách kết hợp nhiều cây để:

Giảm overfitting
Tăng độ chính xác
Làm mô hình ổn định hơn
🧠 Ý tưởng chính — “Nhiều cái đầu hơn một cái đầu!”
Random Forest = Rừng gồm nhiều Decision Tree, mỗi cây được huấn luyện khác nhau một cách ngẫu nhiên, và dự đoán cuối cùng là đa số phiếu. 

🔍 Hai yếu tố "ngẫu nhiên" cốt lõi:
Bootstrap sampling (Bagging):
Mỗi cây được huấn luyện trên một tập con ngẫu nhiên có hoàn lại từ dữ liệu gốc.
Ví dụ: Dữ liệu có 100 mẫu → mỗi cây dùng ~63 mẫu (có lặp).
Random feature selection:
Khi chia một nút, chỉ xét một tập con ngẫu nhiên các đặc trưng (thay vì tất cả).
Ví dụ: Có 10 đặc trưng → mỗi lần chia chỉ xem 3 đặc trưng ngẫu nhiên.
→ Nhờ đó, các cây trong rừng khác nhau, không học cùng một lỗi, nên kết hợp lại sẽ tốt hơn.

💡 Dự đoán như thế nào?
Phân loại: Mỗi cây đưa ra một dự đoán → lớp được nhiều cây chọn nhất thắng (voting).
Hồi quy: Trung bình giá trị dự đoán của tất cả cây.
✅ Ưu điểm:
Rất khó overfit (ngay cả khi cây rất sâu).
Không cần chuẩn hóa dữ liệu.
Xử lý tốt đặc trưng không quan trọng (tự động giảm ảnh hưởng).
Cho điểm quan trọng của đặc trưng (feature importance).
Hiệu suất cao, dễ dùng, ít tinh chỉnh.
❌ Hạn chế:
Ít trực quan hơn Decision Tree đơn (vì là tập hợp nhiều cây).
Dự đoán chậm hơn một cây đơn (nhưng vẫn nhanh hơn KNN!).
Không ngoại suy tốt (giống Decision Tree).


🔹 3. Khi nào dùng Random Forest?
Bạn cần
mô hình mạnh, ít tinh chỉnh
, cho kết quả tốt ngay lập tức
Cần
giải thích chi tiết từng quyết định
(dùng Decision Tree thay vì RF)
Dữ liệu
hỗn hợp
, có nhiễu, có đặc trưng không quan trọng
Bài toán
rất lớn
(triệu mẫu) → hãy cân nhắc
XGBoost/LightGBM
(nhanh hơn)
Muốn
đánh giá đặc trưng nào quan trọng
Cần
xác suất đầu ra rất chính xác
(RF đôi khi cho xác suất bị lệch)
Làm
baseline mạnh
cho hầu hết bài toán có giám sát
Dữ liệu
chuỗi thời gian
có tính thứ tự → RF không tận dụng được

In [1]:
import numpy as np

class SimpleRandomForest:
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features  # số đặc trưng ngẫu nhiên mỗi lần chia
        self.trees = []

    def _bootstrap_sample(self, X, y):
        """Lấy mẫu ngẫu nhiên có hoàn lại (bootstrap)"""
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[idxs], y[idxs]

    def fit(self, X, y):
        self.trees = []
        n_features = X.shape[1]
        # Nếu không chỉ định, mặc định dùng sqrt(n_features) cho phân loại
        if self.max_features is None:
            self.max_features = int(np.sqrt(n_features))

        for _ in range(self.n_trees):
            # Tạo cây mới
            tree = SimpleDecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            
            # Lấy mẫu bootstrap
            X_boot, y_boot = self._bootstrap_sample(X, y)
            
            # Huấn luyện cây (trong phiên bản đầy đủ, bạn sẽ chọn ngẫu nhiên đặc trưng trong _best_split)
            # Ở đây, để đơn giản, ta giả lập bằng cách **giới hạn đặc trưng ngẫu nhiên toàn cây**
            # (thực tế, sklearn chọn ngẫu nhiên **mỗi lần chia**)
            feature_subset = np.random.choice(n_features, size=self.max_features, replace=False)
            X_boot_subset = X_boot[:, feature_subset]
            
            tree.fit(X_boot_subset, y_boot)
            self.trees.append((tree, feature_subset))  # lưu cả cây và đặc trưng đã dùng

    def predict(self, X):
        # Dự đoán từ từng cây
        tree_preds = []
        for tree, features in self.trees:
            X_subset = X[:, features]
            preds = tree.predict(X_subset)
            tree_preds.append(preds)
        
        # Voting: chọn lớp xuất hiện nhiều nhất
        tree_preds = np.array(tree_preds).T  # shape: (n_samples, n_trees)
        predictions = []
        for pred_row in tree_preds:
            most_common = np.bincount(pred_row).argmax()
            predictions.append(most_common)
        return np.array(predictions)

# --- Thử nghiệm ---
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình tự code
rf_model = SimpleRandomForest(n_trees=10, max_depth=4)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"✅ Độ chính xác (Random Forest tự code): {acc:.2%}")

# So sánh với sklearn
from sklearn.ensemble import RandomForestClassifier
sk_rf = RandomForestClassifier(n_estimators=10, max_depth=4, random_state=42)
sk_rf.fit(X_train, y_train)
sk_acc = sk_rf.score(X_test, y_test)
print(f"🔍 Sklearn RandomForest độ chính xác: {sk_acc:.2%}")

# Xem feature importance từ sklearn
print("\n🌟 Feature importance (từ sklearn):")
for name, imp in zip(iris.feature_names, sk_rf.feature_importances_):
    print(f"  {name}: {imp:.3f}")

NameError: name 'SimpleDecisionTree' is not defined